In [28]:
import pathlib, re, numpy as np, pandas as pd, plotly.graph_objects as go

# --- 1.  Paths --------------------------------------------------------------
ROOT_DIR = pathlib.Path(r"C:\Repositories\odi-data-visualization")
DATA_FILE = ROOT_DIR / "data" / "table_3.csv"

PLOTS_DIR = ROOT_DIR / "plots"
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

# --- 2.  Safe filename helper -----------------------------------------------
SAFE  = re.compile(r'[^0-9A-Za-z _()\-.]+')
clean = lambda s, n=80: SAFE.sub("_", str(s))[:n].strip()

# --- 3.  Band colours -------------------------------------------------------
BANDS = [
    (1.00, 3.29, "#e53935"),   # red
    (3.30, 3.49, "#fb8c00"),   # orange
    (3.50, 3.79, "#fdd835"),   # yellow
    (3.80, 5.00, "#43a047"),   # green
]
def band_color(v, default="#999999"):
    for lo, hi, col in BANDS:
        if lo <= v <= hi:
            return col
    return default

# --- 4.  Load table_3 -------------------------------------------------------
df = pd.read_csv(DATA_FILE)

# First col = metric name
metric_names = df.iloc[:, 0].astype(str)
departments = df.columns[1:]  # everything after Outcome

# Ensure numeric
for col in departments:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# --- 5.  Groups -------------------------------------------------------------
ELECTED_DEPTS = {
    "the city auditor", "city auditor",
    "city attorney",
    "city council",
    "treasurers office", "treasurer's office", "treasurer office"
}
def group_of(dept: str) -> str:
    return "elected_officials" if dept.casefold().strip() in ELECTED_DEPTS else "mayors_office"

# --- 6.  Gauge builder ------------------------------------------------------
def build_gauge(score, title):
    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=score,
        title={"text": title, "font": {"size": 18, "color": "#1f3d5a"}},
        number={"font": {"size": 38, "color": "#1f3d5a"}},
        gauge={
            "axis": {
                "range": [1, 5],
                "tick0": 1,          # start at 1
                "dtick": 0.5,        # ticks every 0.5
                "ticks": "outside",
                "ticklen": 8,
                "tickwidth": 2,
                "tickcolor": "#444",
                "tickfont": {"size": 12},
            },
            "bar": {"color": band_color(score), "thickness": 0.45},
            "bgcolor": "#FFFFFF",
            "borderwidth": 1,
            "bordercolor": "#444"
        }
    ))
    fig.update_layout(height=450, width=450,
                      margin=dict(t=60, b=20, l=20, r=20))
    return fig

def save_gauge(fig, dirpath: pathlib.Path, filename: str):
    dirpath.mkdir(parents=True, exist_ok=True)
    fig.write_image(dirpath / f"{filename}.png", scale=2)

# --- 7.  Department-level gauges -------------------------------------------
print("Generating department gauges…")
for dept in departments:
    grp = group_of(dept)
    base_grp_dir = PLOTS_DIR / grp
    dept_dir = base_grp_dir / clean(dept)
    pool_dir = base_grp_dir / "all_pngs"

    for metric, score in zip(metric_names, df[dept]):
        if pd.isna(score):
            continue
        fig = build_gauge(score, f"{dept} — {metric}")
        filename = clean(metric)
        # Save to dept folder
        save_gauge(fig, dept_dir, filename)
        # Save to group all_pngs
        save_gauge(fig, pool_dir, f"{clean(dept)} - {filename}")
        print(f"   • {grp} / {dept} — {metric}")

# --- 8.  Group-level averages ----------------------------------------------
print("\nGenerating group-level gauges…")
for grp in ["elected_officials", "mayors_office"]:
    grp_depts = [d for d in departments if group_of(d) == grp]
    if not grp_depts:
        continue
    base_grp_dir = PLOTS_DIR / grp
    for metric in metric_names:
        scores = df.loc[metric_names == metric, grp_depts].values.flatten()
        scores = scores[~np.isnan(scores)]
        if not len(scores):
            continue
        avg_score = scores.mean()
        fig = build_gauge(avg_score, f"{'Elected Officials' if grp=='elected_officials' else 'Mayor’s Office'} — {metric}")
        save_gauge(fig, base_grp_dir, clean(metric))
        print(f"   • {grp} — {metric}")

print("\nDone.  Plots saved under:")
print("   ", PLOTS_DIR / "elected_officials")
print("   ", PLOTS_DIR / "mayors_office")

Generating department gauges…
   • mayors_office / Board of Health — Overall
   • mayors_office / Board of Health — Training
   • mayors_office / Board of Health — Supervisor Clarity/ Communication
   • mayors_office / Board of Health — Communication with Upper Management
   • mayors_office / Board of Health — Task Significance
   • mayors_office / Board of Health — Autonomy
   • mayors_office / Board of Health — Feedback/Recognition
   • mayors_office / Board of Health — Policies
   • mayors_office / Board of Health — Innovation
   • mayors_office / Board of Health — Satisfaction with my Coworkers/ Department Culture
   • mayors_office / Board of Health — Inter-Department Communication
   • mayors_office / Board of Health — Satisfaction with the Job Itself
   • mayors_office / Board of Health — Work-life Balance
   • mayors_office / Building and Zoning Services — Overall
   • mayors_office / Building and Zoning Services — Training
   • mayors_office / Building and Zoning Services — Su